### Set up

In [2]:
import json
import os

import azure.identity
import dotenv
import numpy as np
import openai
import pandas as pd

# Set up OpenAI client based on environment variables
dotenv.load_dotenv()
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")

azure_credential = azure.identity.DefaultAzureCredential()
token_provider = azure.identity.get_bearer_token_provider(azure_credential,
    "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding
    
def get_embeddings(sentences):
    embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=sentences)
    return [embedding_object.embedding for embedding_object in embeddings_response.data]


### Vector representations

In [6]:
# optimal size to embed is ~512 tokens
vector = get_embedding("im more of a cat person than a dog person but actually these days i just like humans") # 8192 tokens limit

In [7]:
vector # 8192 tokens

[-0.022648831829428673,
 0.006634126417338848,
 -0.02030368708074093,
 -0.02884402684867382,
 -0.009123491123318672,
 0.010998353362083435,
 0.009123491123318672,
 -0.022498341277241707,
 -0.021796051412820816,
 -0.013330956920981407,
 -0.0047561293467879295,
 0.019312957301735878,
 -0.00040032845572568476,
 -0.007029164116829634,
 -0.02124425396323204,
 0.01486094482243061,
 0.028919272124767303,
 -0.011474906466901302,
 0.0103274155408144,
 -0.01764502003788948,
 -0.010365038178861141,
 -0.0030693807639181614,
 0.0015072572277858853,
 -0.028166819363832474,
 -0.011104950681328773,
 0.022824404761195183,
 0.023125385865569115,
 -0.015500529669225216,
 -0.003746588481590152,
 -0.006091732997447252,
 0.015864215791225433,
 -0.012089409865438938,
 0.008628126233816147,
 -0.031026141718029976,
 -0.0017510205507278442,
 -0.012904566712677479,
 -0.004966189153492451,
 -0.0042513590306043625,
 -0.015287335030734539,
 -0.02641109563410282,
 -0.019300416111946106,
 -0.01079142838716507,
 -0.00

In [8]:
len(vector)

1536

### Document similarity modeled as cosine distance

In [10]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

sentences1 = ['The new movie is awesome',
             'The new movie is awesome',
             'The new movie is awesome']

sentences2 = ['djkshsjdkhfsjdfkhsd',
              'This recent movie is so good',
              'The new movie is awesome']

embeddings1 = get_embeddings(sentences1)
embeddings2 = get_embeddings(sentences2)

for i in range(len(sentences1)):
    print(f"{sentences1[i]} \t\t {sentences2[i]} \t\t Score: {cosine_similarity(embeddings1[i], embeddings2[i]):.4f}")

The new movie is awesome 		 djkshsjdkhfsjdfkhsd 		 Score: 0.7467
The new movie is awesome 		 This recent movie is so good 		 Score: 0.9191
The new movie is awesome 		 The new movie is awesome 		 Score: 1.0000


### Vector search

In [11]:
# Load in vectors for movie titles
with open('openai_movies.json') as json_file:
    movie_vectors = json.load(json_file)

In [12]:
# Compute vector for query
query = "My Neighbor Totoro"

embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=[query])
vector = embeddings_response.data[0].embedding

# Compute cosine similarity between query and each movie title
scores = []
for movie in movie_vectors:
    scores.append((movie, cosine_similarity(vector, movie_vectors[movie])))

# Display the top 10 results
df = pd.DataFrame(scores, columns=['Movie', 'Score'])
df = df.sort_values('Score', ascending=False)
df.head(10)

,Movie,Score
221,Toy Story,0.868217
314,Toy Story 2,0.850300
28,The Fox and the Hound,0.848830
534,Kaze Tachinu,0.846195
495,Toy Story 3,0.845041
561,Zootopia,0.842810
365,Sen to Chihiro no Kamikakushi,0.841869
511,Winnie the Pooh,0.841173
1,Pinocchio,0.839548
481,Gake no ue no Ponyo,0.838221
